<a href="https://colab.research.google.com/github/ethancheah-arvato/image-classification/blob/main/ImageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


2.3.1+cu121
True


In [2]:
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers

--2024-08-12 13:38:32--  https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.98.14, 52.217.103.30, 16.182.101.80, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.98.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344873452 (329M) [application/x-gzip]
Saving to: ‘flower_data.tar.gz’

flower_data.tar.gz  100%[===================>] 328.90M  55.3MB/s    in 6.6s    

2024-08-12 13:38:39 (49.5 MB/s) - ‘flower_data.tar.gz’ saved [344873452/344873452]

unlink: cannot unlink 'flowers': No such file or directory


In [3]:
data_dir = '/content/flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [30]:
# Define transforms for the training set
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(45),  # Increased rotation range
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),  # Increased jitter
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transforms for the validation set
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transforms for the testing set
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# Using the image datasets and the transforms, define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

import json

with open('/content/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
# def train_resnet50(train_dataloader, valid_dataloader, epochs=5, print_every=40, learning_rate=0.001):
#     # Load a pre-trained ResNet50 model
#     model = models.resnet50(weights='DEFAULT')  # Updated to use weights argument

#     # Freeze parameters to avoid backpropagation through them
#     for param in model.parameters():
#         param.requires_grad = False

#     # Get the number of input features for the last fully connected layer
#     num_ftrs = model.fc.in_features

#     # Define a new, untrained feed-forward network as a classifier
#     model.fc = nn.Sequential(
#         nn.Linear(2048, 4096),
#         nn.ReLU(),
#         nn.Dropout(0.6),
#         nn.Linear(4096, 1024),
#         nn.ReLU(),
#         nn.Dropout(0.6),
#         nn.Linear(1024, 102),
#         nn.LogSoftmax(dim=1)
#     )

#     # Set the device to GPU if available
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     # Define the criterion and optimizer
#     criterion = nn.NLLLoss()
#     optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

#     # Training loop
#     steps = 0

#     for epoch in range(epochs):
#         model.train()
#         running_loss = 0
#         for inputs, labels in train_dataloader:
#             steps += 1

#             # Move input and label tensors to the GPU
#             inputs, labels = inputs.to(device), labels.to(device)

#             optimizer.zero_grad()

#             # Forward pass
#             logps = model(inputs)
#             loss = criterion(logps, labels)

#             # Backward pass
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()

#             if steps % print_every == 0:
#                 model.eval()
#                 validation_loss = 0
#                 accuracy = 0

#                 with torch.no_grad():
#                     for inputs, labels in valid_dataloader:
#                         inputs, labels = inputs.to(device), labels.to(device)
#                         logps = model(inputs)
#                         batch_loss = criterion(logps, labels)
#                         validation_loss += batch_loss.item()

#                         # Calculate accuracy
#                         ps = torch.exp(logps)
#                         top_p, top_class = ps.topk(1, dim=1)
#                         equals = top_class == labels.view(*top_class.shape)
#                         accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

#                 print(f"Epoch {epoch+1}/{epochs}.. "
#                       f"Train loss: {running_loss/print_every:.3f}.. "
#                       f"Validation loss: {validation_loss/len(valid_dataloader):.3f}.. "
#                       f"Validation accuracy: {accuracy/len(valid_dataloader):.3f}")

#                 running_loss = 0
#                 model.train()


# train_resnet50(train_dataloader, valid_dataloader, epochs=10, print_every=40, learning_rate=0.001)
import copy
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Define the optimizer
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Define a learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

def train_and_evaluate_model(model, dataloaders, criterion, optimizer, device, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            logps = model(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(dataloaders['train'])
        print(f'Epoch {epoch+1}/{num_epochs}.. Train loss: {epoch_loss:.3f}')

        model.eval()
        val_loss = 0
        corrects = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['valid']:
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model(inputs)
                loss = criterion(logps, labels)
                val_loss += loss.item()

                _, preds = torch.max(logps, 1)
                corrects += torch.sum(preds == labels.data)
                total += labels.size(0)

        epoch_acc = corrects.double() / total
        epoch_val_loss = val_loss / len(dataloaders['valid'])
        print(f'Epoch {epoch+1}/{num_epochs}.. Validation loss: {epoch_val_loss:.3f}.. Validation accuracy: {epoch_acc:.3f}')

        # Update learning rate scheduler
        scheduler.step(epoch_val_loss)

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model

# Call the training function
model = train_and_evaluate_model(model, dataloaders, criterion, optimizer, device)

Epoch 1/10.. Train loss: 2.579
Epoch 1/10.. Validation loss: 1.310.. Validation accuracy: 0.671
Epoch 2/10.. Train loss: 2.438
Epoch 2/10.. Validation loss: 1.230.. Validation accuracy: 0.698
Epoch 3/10.. Train loss: 2.378
Epoch 3/10.. Validation loss: 1.218.. Validation accuracy: 0.705


In [34]:
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def test_model(model, test_dataloader, criterion, device):
    model.eval()
    corrects = 0
    total = 0
    test_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = corrects.double() / total
    avg_loss = test_loss / len(test_dataloader)

    print(f'Test Loss: {avg_loss:.3f}')
    print(f'Test Accuracy: {accuracy:.3f}')

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    print(f'Precision: {precision:.3f}')
    print(f'Recall: {recall:.3f}')
    print(f'F1 Score: {f1:.3f}')

    # Compute and plot confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10,7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=dataloaders['test'].dataset.classes,
                yticklabels=dataloaders['test'].dataset.classes)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

# Call the testing function
test_model(model, test_dataloader, criterion, device)

Test Loss: 1.452
Test Accuracy: 0.634
